# DOFs & co

* This notebooks provides some toosl but is also a quick cheat sheet.

A couple of reminders:

* "Local" and "global" qualifiers applied to dofs are not always meant in the same sense. The usual sense is "local to a cell", "global to the function space", but sometimes local seems to mean "local to a process"!
* `dolfin.vertex_to_dof_map()`, `dolfin.dof_to_vertex_map()` only work if all dofs are Lagrange.

In [ ]:
# Boilerplate
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

def __nbinit__():
    """ Initialisation for nbimporter. """
    global __all__
    __all__ = ['plot_dofs', 'extract_dofs_with_mask',
               'list_hermite_dofs', 'list_bubble_dofs']
__nbinit__()

# Plotting and extracting

In [ ]:
def plot_dofs(V:FunctionSpace, dofs:list, **kwargs):
    """ Plots the mesh and the coordinates of dofs in it
    with colored dots.
    
    Arguments
    ---------
        V: FunctionSpace of dimension 1,2 or 3
        dofs: list of indices.
        **kwargs: anything that matplotlib accepts.
    """
    tdim = V.mesh().topology().dim()
    all_dofs = zip(V.dofmap().dofs(), 
                   V.tabulate_dof_coordinates().reshape((-1, tdim)))
    dofs_coordinates = np.array([dof[1] for dof in 
                                 filter(lambda p: p[0] in dofs, all_dofs)])
    plot(V.mesh(), alpha=0.8)

    if dofs_coordinates.size == 0:
        return

    kwargs.setdefault('linewidths', 0)
    kwargs.setdefault('zorder', 10)
    if tdim > 1:
        kwargs.setdefault('s', 15)
        pl.scatter(dofs_coordinates[:,0], dofs_coordinates[:,1], **kwargs)
    else:
        pl.ylim((-0.1, 0.1))
        pl.scatter(dofs_coordinates, np.zeros_like(dofs_coordinates), **kwargs)

In [ ]:
def extract_dofs_with_mask(V:FunctionSpace, mask:list) -> list:
    """ Given a local (per-cell) dof mask, return the list of global dofs which applies.
    E.g. for a function space of elements with 10 dofs, and a mask [1,4,7], return the
    list of all global dof indices corresponding to those local ones at each cell.
    """
    dofs = set()
    dm = V.dofmap()
    for i in range(V.mesh().num_cells()):
        dofs = dofs.union(set(dm.cell_dofs(i)[mask]))
    return list(dofs)

In [ ]:
def list_hermite_dofs(V:FunctionSpace) -> np.ndarray:
    """ Returns the global indices of Hermite dofs in V.

    Arguments
    ---------
        V: Any FunctionSpace. If it has Hermite (or DKT) elements, then
           the indices of those dofs associated to PointDerivatives will
           be returned.
    Returns
    -------
        Global indices of hermite dofs.
    """
    tdim = V.element().topological_dimension()
    e = ffc.fiatinterface.create_element(V.ufl_element())
    
    # This mask filters out the Hermite dofs from the list of dofs of a cell:
    mask = np.array(list(map(lambda f: isinstance(f, FIAT.functional.PointDerivative),
                             e.dual_basis())))

    return np.array(extract_dofs_with_mask(V, mask), dtype=np.uintc)

In [ ]:
def list_bubble_dofs(V:FunctionSpace) -> np.ndarray:
    """ Returns a list of all bubble dofs in V.
    This only handles a couple of cases that I've needed.
    """
    family = V.ufl_element().family().lower()
    tdim = V.ufl_cell().topological_dimension()
    degree = V.ufl_element().degree()
    if family == 'hermite' and tdim == 2:
        mask = [-1]
    elif family == 'lagrange' and tdim == 2 and degree == 3:
        mask = [-1]

    return np.array(extract_dofs_with_mask(V, mask), dtype=np.uintc)

In [ ]:
V = FunctionSpace(UnitSquareMesh(5,5), "Lagrange", 3)
plot_dofs(V, list_bubble_dofs(V))

# Pot-pourri

In [ ]:
V = FunctionSpace(UnitIntervalMesh(4), "Lagrange", 1)
u = interpolate(Expression("x[0]*x[0]", degree=1), V)

# These only work on spaces where every vertex has exactly one dof
dof_to_vertex_map(V)
vertex_to_dof_map(V)

# How to get expansion coefs on cell.
# An alternative to this is u.restrict()
cell_id = 1
dofs = V.dofmap().cell_dofs(cell_id)
dofs = u.vector()[dofs]

# Global to local mapping (not needed):

dm = V.dofmap()
g2l = {}
for cell in range(V.mesh().num_cells()):
    l2g = dm.cell_dofs(cell)
    g2l.update({v:k for k,v in zip(range(len(l2g)), l2g)})

In [ ]:
mesh = refine(refine(refine(UnitTriangleMesh())))
cell_id = mesh.bounding_box_tree().compute_first_entity_collision(Point(np.array([0.1, 0.1])))
cell = Cell(mesh, cell_id)

# The following two should coincide if all dofs are on vertices.
print("Cell.get_coordinate_dofs() returns the physical coordinates associated to the DOFs in the cell.")
print(cell.get_coordinate_dofs())
print("Cell.get_vertex_coordinates() returns the physical coordinates of the vertices in the cell.")
print(cell.get_vertex_coordinates())

In [ ]:
def tabulate_cell_data(V):
    """ Print some info on dofs and coords of cells in a mesh.

    Arguments:
    ----------
        V: FunctionSpace.
    """
    mesh = V.mesh()
    element = V.element()
    dm = V.dofmap()
    for cell in cells(mesh):
        print("At cell: %s" % cell)
        print("\tDOF coordinates: %s" % np.round(element.tabulate_dof_coordinates(cell), 3).T)
        l2g = ["%d -> %d" % (i, j) for i,j in enumerate(dm.cell_dofs(cell.index()))]
        print("\tLocal to global DOF map: %s" % ", ".join(l2g))
        print("\tCell coordinate DOFs: %s" % cell.get_coordinate_dofs())

In [ ]:
tabulate_cell_data(V)

# Dofs in VectorFunctionSpace&lt;CG2&gt;

The first six are for one component, the second six for the second one (i.e. they go in pairs (n, n+6)). It seems that the ordering is $z_1, z_2, z_3$ then $s_1, s_2, s_3$ with $s_l$ on the side facing $z_l$. ** CHECK THIS **

In [ ]:
mesh = UnitSquareMesh(1, 1)
V = FunctionSpace(mesh, 'DKT', 3)
Vp = VectorFunctionSpace(mesh, "Lagrange", 2, dim=2)
dm = V.dofmap()
dmp1, dmp2 = Vp.sub(0).dofmap(), Vp.sub(1).dofmap()

coords = Vp.tabulate_dof_coordinates().reshape((-1,2))
for i in range(2):
    print("=====Cell %d=====" % i)
    for dof, xy in zip(dmp1.cell_dofs(i), coords[dmp1.cell_dofs(i)]):
        print("%d -> %s" %(dof, xy))

plot_dofs(Vp, dmp1.cell_dofs(0))

We need to remap the indices to have both components for each dof consecutively listed:

In [ ]:
from itertools import chain

dofs = list(chain.from_iterable(zip(dmp1.cell_dofs(0), dmp2.cell_dofs(0))))
dofs